<a href="https://colab.research.google.com/github/lawrennd/qig-code/blob/main/examples/lme_numeric_symbolic_bridge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bridging Symbolic and Numeric LME Representations

This notebook illustrates how the **symbolic** LME (Locally Maximally Entangled) decomposition in `qig.symbolic.lme_exact` connects to the **numeric** quantum exponential family in `qig.exponential_family`.

## Key Concepts

1. **LME states** are globally pure but locally maximally mixed. For a qutrit pair (d=3), the maximally entangled state has the form:
   $$|\Phi^+\rangle = \frac{1}{\sqrt{3}}\sum_{i=0}^{2}|ii\rangle$$

2. **Regularised Bell states**: Pure states have infinite natural parameters (since $\log 0 = -\infty$). We regularize by mixing with the maximally mixed state:
   $$\rho_\epsilon = (1-\epsilon)|\Phi^+\rangle\langle\Phi^+| + \frac{\epsilon}{9}I_9$$

3. **Block structure**: At the LME origin, the density matrix has a special block structure when reordered to the "symbolic basis":
   - A 3×3 entangled block (rank-1 in the pure case)
   - Three 2×2 blocks
   - Three 1×1 diagonal entries

4. **Natural parameters**: The exponential family parametrization $\rho(\theta) = \exp(K(\theta))/Z(\theta)$ where $K(\theta) = \sum_a \theta_a F_a$.

In [ ]:
import numpy as np
from scipy.linalg import logm
import matplotlib.pyplot as plt

# Numeric exponential family
from qig.exponential_family import QuantumExponentialFamily

# Symbolic LME tools
from qig.symbolic.lme_exact import (
    numeric_lme_blocks_from_theta,
    permutation_matrix,
    extract_blocks
)
from qig.pair_operators import pair_basis_generators

## 1. Construct a Numeric Regularized Bell State

We use `QuantumExponentialFamily` with `pair_basis=True` for a qutrit pair (d=3). The `get_bell_state_parameters` method returns the natural parameters $\theta$ for a regularized Bell state.

Using `log_epsilon` allows us to approach the pure-state boundary without numerical issues.

In [ ]:
# Create a qutrit-pair exponential family
qef = QuantumExponentialFamily(n_pairs=1, d=3, pair_basis=True)
print(f"Dimension: d = {qef.d}")
print(f"Number of operators: {len(qef.operators)}")
print(f"Hilbert space dimension: {qef.operators[0].shape[0]}")

# Get Bell state parameters with log_epsilon for numerical stability
log_epsilon = -20  # Very close to pure state
theta_bell = qef.get_bell_state_parameters(log_epsilon=log_epsilon)

# Compute entropy separately
rho_bell = qef.rho_from_theta(theta_bell)
entropy = -np.trace(rho_bell @ logm(rho_bell)).real

print(f"\nlog_epsilon = {log_epsilon}")
print(f"Effective epsilon ≈ {np.exp(log_epsilon):.2e}")
print(f"Max |theta|: {np.max(np.abs(theta_bell)):.4f}")
print(f"Entropy: {entropy:.6e}")

## 2. Map to the Symbolic Block Basis

The key insight from `qig.symbolic.lme_exact` is that when we reorder the computational basis to the "entanglement basis", the density matrix (and $K(\theta) = \log\rho + \text{const}$) takes a block-diagonal form:

**Block structure for d=3:**
- Indices `{0,4,8}` → 3×3 entangled block (the "heart" of the Bell state)
- Indices `{1,3}`, `{2,6}`, `{5,7}` → three 2×2 blocks
- Indices remaining → 1×1 diagonal entries

The `numeric_lme_blocks_from_theta` helper transforms numeric $\theta$ into this block basis.

In [ ]:
# Extract blocks from theta
blocks = numeric_lme_blocks_from_theta(theta_bell, qef.operators)

print("=== Extracted Blocks from K(theta) ===\n")

print("3x3 Entangled Block:")
print(blocks['ent_3x3'])
print()

print("2x2 Block:")
print(blocks['block_2x2'])
print()

print("1x1 Diagonal entries:")
print(f"  diag_1: {blocks['diag_1']}")
print(f"  diag_2: {blocks['diag_2']}")
print(f"  diag_3: {blocks['diag_3']}")
print(f"  diag_4: {blocks['diag_4']}")

## 3. Understanding the Eigenvalue Structure at LME

### Why does the Bell state have this block structure?

The maximally entangled state $|\Phi^+\rangle$ lives entirely in the **symmetric subspace** of the two-qutrit system. When we write it in the computational basis:
$$|\Phi^+\rangle = \frac{1}{\sqrt{3}}(|00\rangle + |11\rangle + |22\rangle)$$

The corresponding density matrix $\rho = |\Phi^+\rangle\langle\Phi^+|$ has:
- Non-zero entries only at positions $(i \cdot d + i, j \cdot d + j)$ for $i,j \in \{0,1,2\}$
- These are exactly indices $\{0, 4, 8\}$ in a 9×9 matrix

### Eigenvalues of the regularized Bell state

For $\rho_\epsilon = (1-\epsilon)|\Phi^+\rangle\langle\Phi^+| + \frac{\epsilon}{d^2}I$:

| Eigenvalue | Multiplicity | Origin |
|------------|--------------|--------|
| $(1-\epsilon) + \epsilon/9$ | 1 | The Bell state direction |
| $\epsilon/9$ | 8 | Orthogonal directions |

As $\epsilon \to 0$, one eigenvalue → 1 and the rest → 0.

In [ ]:
# Verify eigenvalue structure numerically
rho = qef.rho_from_theta(theta_bell)
eigenvalues = np.linalg.eigvalsh(rho)
eigenvalues_sorted = np.sort(eigenvalues)[::-1]  # Descending

print("=== Eigenvalue Verification ===\n")
print("Computed eigenvalues (descending):")
for i, ev in enumerate(eigenvalues_sorted):
    print(f"  λ_{i} = {ev:.10e}")

# Analytic expectations
eps = np.exp(log_epsilon)
lambda_max_analytic = (1 - eps) + eps/9
lambda_rest_analytic = eps/9

print(f"\nAnalytic predictions for ε = {eps:.2e}:")
print(f"  λ_max  = (1-ε) + ε/9 = {lambda_max_analytic:.10e}")
print(f"  λ_rest = ε/9         = {lambda_rest_analytic:.10e}")

print(f"\nRelative errors:")
print(f"  |λ_0 - λ_max|/λ_max = {abs(eigenvalues_sorted[0] - lambda_max_analytic)/lambda_max_analytic:.2e}")
print(f"  |λ_1 - λ_rest|/λ_rest = {abs(eigenvalues_sorted[1] - lambda_rest_analytic)/lambda_rest_analytic:.2e}")

## 4. Structure of $\log(\rho_\epsilon)$ and the Natural Parameters

The natural parameters $\theta$ encode the structure of $K = \log\rho + \psi(\theta)I$ (up to the normalizing constant).

### Log-eigenvalues

Taking the logarithm of the eigenvalues:
- $\log\lambda_{max} = \log((1-\epsilon) + \epsilon/9) \approx \log(1) = 0$ for small $\epsilon$
- $\log\lambda_{rest} = \log(\epsilon/9) = \log\epsilon - \log 9$

The **gap** between these log-eigenvalues grows as $|\log\epsilon|$, which is why $|\theta|_{max}$ scales like $|\log\epsilon|$.

In [ ]:
# Analyze log-eigenvalue structure
log_eigenvalues = np.log(eigenvalues_sorted)

print("=== Log-Eigenvalue Analysis ===\n")
print("log(eigenvalues):")
for i, lev in enumerate(log_eigenvalues):
    print(f"  log(λ_{i}) = {lev:.4f}")

gap = log_eigenvalues[0] - log_eigenvalues[1]
print(f"\nGap: log(λ_0) - log(λ_1) = {gap:.4f}")
print(f"Expected gap ≈ -log_epsilon + log(9) = {-log_epsilon + np.log(9):.4f}")

print(f"\nThis gap explains why max|θ| ≈ {np.max(np.abs(theta_bell)):.4f}")

## 5. Connection to Symbolic Parameters 'a' and 'c'

In the symbolic decomposition (`qig.symbolic.lme_exact`), the 3×3 entangled block is parametrized by:
- `a`: Controls the "diagonal spread" (related to how eigenvalues separate)
- `c`: An off-diagonal coupling parameter

At the LME origin (maximally entangled state), the block has a specific structure determined by the Bell state projector.

In [ ]:
# Analyze the 3x3 entangled block
block_3x3 = blocks['ent_3x3']

print("=== 3x3 Entangled Block Analysis ===\n")
print("Block K_entangled:")
print(block_3x3)

# The block should be close to a rank-1 projector (in exponentiated form)
# For the Bell state: K_entangled ≈ log_lambda_max * |1,1,1><1,1,1|/3 + log_lambda_rest * (I - projector)

# Check eigenvalues of the block
block_eigs = np.linalg.eigvalsh(block_3x3)
print(f"\nEigenvalues of 3x3 block: {block_eigs}")

# The gap in the block should match the overall gap
block_gap = np.max(block_eigs) - np.min(block_eigs)
print(f"Gap in block eigenvalues: {block_gap:.4f}")

## 6. Scaling Behavior: How $|\theta|$ Grows as $\epsilon \to 0$

The natural parameters grow logarithmically as we approach the pure state:
$$|\theta|_{\max} \sim |\log\epsilon|$$

This is the "mild singularity" mentioned in the paper—the divergence is only logarithmic, not polynomial or exponential.

In [ ]:
# Demonstrate scaling of theta with log_epsilon
log_eps_values = [-5, -10, -20, -50, -100, -200]
theta_maxes = []
entropies = []

print("=== Scaling Analysis ===\n")
print(f"{'log_ε':>10} {'max|θ|':>12} {'Entropy':>15} {'|max|θ|/|log_ε|':>18}")
print("-" * 60)

for le in log_eps_values:
    theta_tmp = qef.get_bell_state_parameters(log_epsilon=le)
    theta_max = np.max(np.abs(theta_tmp))
    theta_maxes.append(theta_max)

    # Compute entropy
    rho_tmp = qef.rho_from_theta(theta_tmp)
    H = -np.trace(rho_tmp @ logm(rho_tmp)).real
    entropies.append(H)

    ratio = theta_max / abs(le)
    print(f"{le:>10} {theta_max:>12.4f} {H:>15.6e} {ratio:>18.4f}")

# Plot the scaling
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(np.abs(log_eps_values), theta_maxes, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('|log ε|')
axes[0].set_ylabel('max |θ|')
axes[0].set_title('Natural Parameter Scaling')
axes[0].grid(True, alpha=0.3)

axes[1].semilogy(np.abs(log_eps_values), entropies, 'ro-', linewidth=2, markersize=8)
axes[1].set_xlabel('|log ε|')
axes[1].set_ylabel('Entropy')
axes[1].set_title('Entropy vs Regularization')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nThe ratio max|θ|/|log_ε| converges to ≈ {theta_maxes[-1]/abs(log_eps_values[-1]):.4f}")

## Summary

This notebook demonstrates the bridge between:

1. **Numeric representation** (`QuantumExponentialFamily`):
   - Natural parameters $\theta \in \mathbb{R}^{80}$ for qutrit pairs
   - Full 9×9 density matrices
   - Computed via matrix logarithm and projection onto basis operators

2. **Symbolic representation** (`lme_exact`):
   - Block-diagonal structure in the entanglement basis
   - Parameters 'a', 'c' controlling the 3×3 entangled block
   - Exact analytic expressions possible in this basis

**Key insights:**
- The LME origin has special structure due to the Bell state's symmetry
- Natural parameters diverge logarithmically as entropy → 0
- The `log_epsilon` parametrization provides numerically stable access to near-pure states
- Block decomposition reveals the "geometric heart" of entanglement